# 第 7 章：RNA-Seq カウントデータの処理
----

## 7.1 準備

### 7.1.1 RNA-Seq とは
RNA-Seq とは、mRNA や miRNA の配列をシークエンスして、発現量の定量や新規転写産物の同定を行う手法である。シークエンスで得られたデータ（リード）は、次のステップで解析する。
1. リードのトリミング
2. ゲノム配列へのマッピング
3. マッピングされたリード数を数える
4. サンプルごとの総リード数の違いや、遺伝子配列長の違いを補正（正規化）
5. 遺伝子ごとの発現量を同定、比較

この章では上記ステップのうち 4 以降を扱い、主に pandas を使って処理を行う。

### 7.1.2 この章で用いる RNA-Seq データ
この章では、異なる 2 条件で培養された酵母 (Saccharomyces cerevisiae) の RNA-Seq データを使用する。この RNA-Seq データは、DDBJ の SRA データベースにアクセッション番号 SRR453566 から SRR453571 の 6 つのデータとして存在している。
- SRR453566, SRR453567, SRR453568 : 通常のグルコース濃度培地による Batch 培養のサンプルから得られた
- SRR453569, SRR453570, SRR453571 : 低グルコース濃度を維持した培地による Chemostat 培養のサンプルから得られた

いずれも Biological triplicates になっている。

### 7.1.3 本章で使用するデータファイル
- counts.txt
- gene_id_product.tsv

## 7.2 データファイルの読み込みとアノテーション
### 7.2.1 カウントデータ

In [ ]:
# 7-2 : 各種インポート
import pandas as pd
import numpy as np

In [ ]:
# 7-3 : ファイルのパス指定
count_file = 'counts.txt'
gene_id_product_file = 'gene_id_product.tsv'

In [ ]:
# 7-4 : ファイル読み込み
df = pd.read_table(count_file, index_col=0, skiprows=1)

- skiprows : DataFrame に読み込まない行 (ファイルの先頭行を 0 として数える)を数字で指定する。リストで指定することもできる
  - `skiprows=1` の代わりに `header=1` としても良い
- index_col : インデックスとして用いる列を数字で指定する

### 7.2.2 データの概観

In [ ]:
# 7-5 : 読み込んだファイルの確認
df.head()

In [ ]:
# 7-6 : 特定の列 (Start の列) だけ確認
df.Start.head()

In [ ]:
# 7-7 : ファイルの行数と列数の確認
df.shape

### 7.2.3 列名を変更する

In [ ]:
# 7-8 : 列名を変更するための辞書の宣言
names = {'SRR453566.sorted.bam': 'batch_1',
         'SRR453567.sorted.bam': 'batch_2',
         'SRR453568.sorted.bam': 'batch_3',
         'SRR453569.sorted.bam': 'chemostat_1',
         'SRR453570.sorted.bam': 'chemostat_2',
         'SRR453571.sorted.bam': 'chemostat_3',}

In [ ]:
# 7-9 : 列名の変更
df = df.rename(mapper=names, axis=1)

In [ ]:
# 7-10 : 変更できたか確認
df.head()

### 7.2.4 ミトコンドリア上の遺伝子を除く

In [ ]:
# 7-11 : ミトコンドリア上の遺伝子に一致しないものだけ取り出す
df = df[df.Chr != 'NC_001224.1']

In [ ]:
# 7-12 : 処理できたか確認 (正しければデータ件数は 6,394 件)
df.shape

### 7.2.5 アノテーションファイルの読み込み

In [ ]:
# 7-13 : ファイルの読み込み
gene_products = pd.read_table(gene_id_product_file, index_col=0, names=['gene_id', 'product'])

ヘッダー名を左から `gene_id`, `product` とする。

In [ ]:
# 7-14 : ファイルの確認
gene_products.head()

### 7.2.6 カウントデータと description を連結する

In [ ]:
# 7-15 カウントデータとアノテーションファイルの結合
df_with_product = gene_products.join(df)